# Using InterSystems Vector Search with LlamaIndex

In this notebook, we'll leverage the Vector Search capabilities available in [InterSystems IRIS 2024.1](https://www.intersystems.com/news/iris-vector-search-support-ai-applications/) and [InterSystems IRIS Cloud SQL](https://developer.intersystems.com/products/iris-cloud-sql-integratedml/), using the well-known [LlamaIndex](https://www.llamaindex.ai/) framework.

## Setting up the connection

First, let's make sure we set up the connection to your InterSystems IRIS instance or Cloud SQL deployment. When targeting a Cloud SQL deployment, change the username and password to `SQLAdmin` and the corresponding password you chose when creating the deployment, and set the port to 443. 


In [ ]:
import os

username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = 1972 
namespace = 'USER'

### Securing the connection

If the target you're connecting to requires secure connections, as is the case for Cloud SQL deployments, we need to supply a certificate and some additional settings to the driver. For Cloud SQL, you can download the certificate file from your deployment's details screen. Look for the button that says "Get X.509 certificate", and copy it into a local folder, such as `/usr/cert-demo/`. If you're running this notebook in a container, you can copy the certificate file into the container using the following command:

```Shell
docker cp ~/Downloads/certificateSQLaaS.pem iris-vector-search-jupyter-1:/usr/cert-demo/certificateSQLaaS.pem
```

Remember to also set the port to 443 in the cell above.

In [ ]:
import ssl

certificateFile = "/usr/cert-demo/certificateSQLaaS.pem"

if (os.path.exists(certificateFile)):
    print("Located SSL certficate at '%s', initializing SSL configuration", certificateFile)
    sslcontext = ssl.create_default_context(cafile=certificateFile)
else:
    print("No certificate file found, continuing with insecure connection")
    sslcontext = None

In [ ]:
from sqlalchemy import create_engine, text

url = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

engine = create_engine(url, connect_args={"sslcontext": sslcontext})
with engine.connect() as conn:
    print(conn.execute(text("SELECT 'hello world!'")).first()[0])

## Creating Vectors using LlamaIndex

In the following cell we'll leverage standard LlamaIndex components to read the files in the `/data/paul_graham/` directory and prepare them for creating embeddings.

In [1]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("First Document ID:", documents[0].doc_id)

### Setting up your OpenAI API key

If you have an OpenAI subscription, use the following cell to pick up your OpenAI API key, and use `OpenAIEmbeddings()` in the cells below. 

Alternatively, you can skip this step and use a local embeddings model that's included in the libraries already imported, such as `HuggingFaceEmbeddings()`, `FastEmbeddings()`, or `FakeEmbeddings()` (for testing purposes!). Just comment / uncomment the corresponding lines in the cells further down the notebook.

In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

The next cell sets up a local language model as the default to create the embeddings if you don't have an OpenAI key. LlamaIndex' default is to use OpenAI, so not running the following cell will assume you want to continue with OpenAI.

In [ ]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import FakeEmbeddings

if not os.environ.get("OPENAI_API_KEY"):
    lc_embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # lc_embed_model = FakeEmbeddings(size=1536)

    # ServiceContext captures how vectors will be generated
    service_context = ServiceContext.from_defaults(
        embed_model=LangchainEmbedding(lc_embed_model), 
        llm=None
    )

    set_global_service_context(service_context)

Now we'll configure the VectorStore object that will be used to save our vectors in IRIS.

In [5]:
from llama_index import StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

# StorageContext captures how vectors will be stored
vector_store = IRISVectorStore.from_params(
    connection_string = url,
    table_name = "paul_graham_essay",
    embed_dim = 1536,  # openai embedding dimension
    engine_args = { "connect_args": {"sslcontext": sslcontext} }
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

And now putting it all together: feeding the documents to our VectorStore.

In [6]:
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
)
query_engine = index.as_query_engine()

/Users/aryanput/anaconda3/envs/iris-vector-search/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 22/22 [00:00<00:00, 39.46it/s]


In [7]:
# # If reconnecting to the vector store, use this: 

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# query_engine = index.as_query_engine()

# # Adding documents to existing index

# for d in documents:
#     index.insert(document=d, storage_context=storage_context)

In [8]:
response = query_engine.query("What did the author do?")

In [9]:
import textwrap
print(textwrap.fill(str(response), 100))

The author worked on writing essays and programming before college. Initially, the author wrote
short stories and experimented with programming on an IBM 1401 using Fortran. Later, with the
introduction of microcomputers, the author's interest in programming grew. The author's first
personal computer was a TRS-80, on which they wrote simple games, a rocket flight prediction
program, and a word processor. In college, the author initially planned to study philosophy but
eventually switched to studying AI due to the influence of a novel and a PBS documentary.


In [10]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

AI was in the air in the mid 1980s, and two specific events influenced the individual's interest in
working on it.
